## Libraries executar antes do load

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

## Preprocessing the Training Set executar antes do load

In [4]:
train_set = tf.keras.utils.image_dataset_from_directory(
    'dataset/training_set/',
    image_size=(64, 64),
    batch_size=32,
    label_mode='binary'  # Configura saída categórica
)

Found 8000 files belonging to 2 classes.


## Preprocessing the Test Set executar antes do load

In [6]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'dataset/test_set/',
    image_size=(64, 64),
    batch_size=32,
    label_mode='binary'  # Configura saída categórica
)

Found 2000 files belonging to 2 classes.


In [7]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_set = train_set.map(lambda x, y: (normalization_layer(x), y))
test_set = test_set.map(lambda x, y: (normalization_layer(x), y))

## Convolution Pooling Flattening

In [9]:
cnn = tf.keras.models.Sequential([
    
    tf.keras.layers.Input(shape=(64, 64, 3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    
    # pool_size janelas 2x2 que analisa, strides de quanto em quanto se passa na imagem
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    # Segunda camada
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    
    # Camada de achatamento (flatten) para transformar os dados 2D em um vetor 1D
    tf.keras.layers.Flatten(),
    
    # Camada totalmente conectada com 128 unidades e ativação ReLU
    tf.keras.layers.Dense(units=128, activation='relu'),
    
    # Camada de saída com 1 unidade e ativação sigmoide para classificação binária
    tf.keras.layers.Dense(units=1, activation='sigmoid')

])

## Traning the CNN

In [11]:
cnn.compile( optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.5622 - loss: 0.6795 - val_accuracy: 0.6985 - val_loss: 0.5943
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.6958 - loss: 0.5792 - val_accuracy: 0.7245 - val_loss: 0.5504
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7459 - loss: 0.5164 - val_accuracy: 0.7435 - val_loss: 0.5203
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.7807 - loss: 0.4587 - val_accuracy: 0.7465 - val_loss: 0.5340
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8153 - loss: 0.4055 - val_accuracy: 0.7515 - val_loss: 0.5433
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8469 - loss: 0.3531 - val_accuracy: 0.7345 - val_loss: 0.6202
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8864 - loss: 0.2743 - val_accuracy: 0.7510 - val_loss: 0.6628
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9106 - loss: 0.2161 - val_acc

In [13]:
cnn.save('modelo_gato_cachorro.keras')

## Single Prediction

In [15]:
cnn = load_model('modelo_gato_cachorro.keras', compile=False)